In [1]:
%load_ext watermark
%watermark

Last updated: 2021-05-14T21:00:38.960002-05:00

Python implementation: CPython
Python version       : 3.7.6
IPython version      : 7.12.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [22]:
import pandas as pd
import numpy as np
variable_objetivo = "ventas"

movies = pd.read_csv("data/movies.csv")

In [3]:
movies.head()

,franquicia,presupuesto,genero,pagina_web,imdb_id,lenguaje,popularidad,productores,pais,fecha_estreno,ventas,duracion,estado,titulo,puntuacion,n_votos
0,Father of the Bride Collection,NaN,Comedy,NaN,tt0113041,en,8.387519,Sandollar Productions,United States of America,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,5.7,173.0
1,NaN,NaN,Drama,NaN,tt0112749,en,0.894647,Miramax,South Africa,1995-12-15,676525.0,106.0,Released,"Cry, the Beloved Country",6.7,13.0
2,Friday Collection,3500000.0,Comedy,http://www.newline.com/properties/friday.html,tt0113118,en,14.569650,New Line Cinema,United States of America,1995-04-26,28215918.0,91.0,Released,Friday,7.0,513.0
3,NaN,NaN,Comedy,NaN,tt0115697,en,8.963037,Paramount Pictures,United States of America,1996-02-01,32.0,87.0,Released,Black Sheep,6.0,124.0
4,NaN,12000000.0,Comedy,NaN,tt0116483,en,9.592265,Universal Pictures,United States of America,1996-02-16,41205099.0,92.0,Released,Happy Gilmore,6.5,767.0


In [4]:
from sklearn import preprocessing, feature_extraction

Este paso es necesario en la versión actual de sklearn 

In [5]:
class BinarizadorCategorico(preprocessing.LabelBinarizer):
    def fit(self, X, y=None):
        super(BinarizadorCategorico, self).fit(X)
        
    def transform(self, X, y=None):
        return super(BinarizadorCategorico, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(BinarizadorCategorico, self).fit(X).transform(X)
    
    
class CodificadorCategorico(preprocessing.LabelEncoder):
    def fit(self, X, y=None):
        super(CodificadorCategorico, self).fit(X)
        
    def transform(self, X, y=None):
        return super(CodificadorCategorico, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(CodificadorCategorico, self).fit(X).transform(X)

En primer lugar cambiamos el imputador que ya no se encuentra en el modulo preprocessing 

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.impute import SimpleImputer

en lugar de buscar mediante dtypes y escribir las colunas numericas, es más sencillo seleccionar mediante una selección directa de dtypes

In [8]:
columnas_numericas = movies.select_dtypes(np.number).columns
columnas_numericas

Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object')

El transformador elimina todas las columnas que no estoy requiriendo 

In [9]:
from sklearn.base import TransformerMixin, BaseEstimator

class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns, output_type="matrix"):
        self.columns = columns
        self.output_type = output_type

    def transform(self, X, **transform_params):
        if isinstance(X, list):
            X = pd.DataFrame.from_dict(X)
        if self.output_type == "matrix":
            return X[self.columns].values
        elif self.output_type == "dataframe":
            return X[self.columns]
        raise Exception("output_type tiene que ser matrix o dataframe")

    def fit(self, X, y=None, **fit_params):
        return self

In [11]:
pipeline_procesado = make_pipeline(
    ColumnExtractor(columns=columnas_numericas),
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

In [12]:
pipeline_procesado

Pipeline(memory=None,
         steps=[('columnextractor',
                 ColumnExtractor(columns=Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object'),
                                 output_type='matrix')),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True))],
         verbose=False)

In [13]:
pipeline_procesado

Pipeline(memory=None,
         steps=[('columnextractor',
                 ColumnExtractor(columns=Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object'),
                                 output_type='matrix')),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True))],
         verbose=False)

### Transforma el dataset. Imprime las dimensiones del output.

In [16]:
movies_procesada = pipeline_procesado.fit_transform(movies)
movies_procesada.shape

(1349, 6)

In [17]:
movies_procesada

array([[ 2.31225306e-16,  4.86394864e-01,  6.76155962e-01,
         1.85918427e-01, -2.28649693e-01, -1.73928134e-01],
       [ 2.31225306e-16, -9.32548795e-01, -5.82358615e-01,
         1.85918427e-01,  4.50154083e-01, -4.99793331e-01],
       [-8.78176892e-01,  1.65712027e+00, -1.25736243e-01,
        -4.24055340e-01,  6.53795216e-01,  5.18535411e-01],
       ...,
       [ 2.31225306e-16, -1.08102798e+00,  1.23535830e-16,
        -6.27379929e-01, -2.28649693e-01, -5.20159906e-01],
       [ 2.31225306e-16,  7.42911802e-01,  1.23535830e-16,
        -4.64720258e-01, -2.28649693e-01, -3.99997114e-01],
       [ 2.31225306e-16, -1.07887709e+00,  1.23535830e-16,
        -4.12456286e+00, -2.50085602e-02, -5.24233221e-01]])

### Crea un pipeline igual que el anterior pero que además tenga un estimador LinearRegression. Para la variable objetivo (ventas), calcula el MAE (mean absolute error) de este estimador (usando el pipeline)

In [18]:
columnas_numericas

Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object')

In [23]:
columnas_numericas_independientes = movies.drop(columns=variable_objetivo).select_dtypes(
                                                                            np.number).columns
columnas_numericas_independientes

Index(['presupuesto', 'popularidad', 'duracion', 'puntuacion', 'n_votos'], dtype='object')

In [25]:
movies[movies[variable_objetivo].isnull()].shape

(141, 16)

Encontramos que hay valores nulos en la variable objetivo que no pueden ser predichos, por lo tanto trabajaremos cun un data set más pequeño que no tenga valores nulos

In [26]:
movies_sales = movies[movies[variable_objetivo].notnull()]

In [27]:
movies_sales.shape

(1208, 16)

In [28]:
from sklearn.linear_model import LinearRegression

estimador = LinearRegression()

pipeline_procesado = make_pipeline(
    ColumnExtractor(columns=columnas_numericas_independientes),
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

pipeline_estimador = make_pipeline(
    pipeline_procesado,
    estimador
)

In [32]:
from sklearn.model_selection import cross_val_score

resultados = cross_val_score(pipeline_estimador,
                            X=movies_sales,
                            y=movies_sales[variable_objetivo],
                            scoring="neg_mean_absolute_error",
                            cv=3)

In [33]:
resultados

array([-27754391.23595949, -22899949.06327263, -22451033.56074178])

In [34]:
resultados.mean()

-24368457.953324635

Hay una diferencia de 24 millones entre lo predicho y lo real 